# IBM Data Science - Professional Certificate
## Capstone Projects - Battle of Neighborhoods
### Author: Vitor Villas Boas

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [ ]:
CLIENT_ID = 'BRQRODSZVFCG1SGCRV5IBOH3VGD0TDDJUCNTP3HW2Q23XEKE' # your Foursquare ID
CLIENT_SECRET = 'OPQVXCAULA3B0SCVMMK1KNEKCXSFMGIE40C4UKTB1IFVGK5Q' # your Foursquare Secret
VERSION = '20180604'
ACCESS_TOKEN = 'AIMBO2C1MFQ1P03XG3ZI3JRL3Z22MOEHZYOCEMZMYI325RJ4' # your access token to user.id
RADIUS = 10000
LIMIT = 100

In [ ]:
CITY = "Sao Paulo, SP"
url_SP = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&oauth_token={}&v={}&radius={}&limit={}&near={}".format(
    CLIENT_ID,
    CLIENT_SECRET,
    ACCESS_TOKEN,
    VERSION,
    RADIUS,
    LIMIT,
    CITY)
results = requests.get(url_SP).json()

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues) # flatten JSON
# nearby_venues.head()

# filter columns
filtered_columns = ['venue.id', 'venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print(nearby_venues.shape)
nearby_venues.head(10)

(100, 5)


,id,name,categories,lat,lng
0,4b17eb00f964a520a1c923e3,Centro Cultural Banco do Brasil (CCBB),Cultural Center,-23.547588,-46.634683
1,573dab3f498e4e0d56c67f4e,Casa de Francisca,Music Venue,-23.548733,-46.634763
2,4f29bb59e4b09b0140fba336,Teatro CCBB,Theater,-23.547644,-46.634733
3,4bdafa28c79cc928177a80e9,Theatro Municipal de São Paulo,Theater,-23.545477,-46.638812
4,4b09acf9f964a520701b23e3,Teatro Renault,Theater,-23.554120,-46.638695
5,4b05bc5ef964a520fee122e3,Mercado Municipal Paulistano,Market,-23.541673,-46.629742
6,560ee563498e447aba686327,A Casa do Porco,Brazilian Restaurant,-23.544887,-46.644622
7,54ac1385498e545686d23f88,Por um Punhado de Dólares,Coffee Shop,-23.548488,-46.645305
8,58f278f8e65d0c4b18f857f0,Kenzo Sushi,Sushi Restaurant,-23.557705,-46.636036
9,4b0588c7f964a520e8d922e3,Pinacoteca do Estado de São Paulo,Art Museum,-23.534735,-46.634001


In [ ]:
results['response']['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'reasonName': 'globalInteractionReason',
     'summary': 'This spot is popular',
     'type': 'general'}]},
  'referralId': 'e-0-4b17eb00f964a520a1c923e3-0',
  'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_',
      'suffix': '.png'},
     'id': '52e81612bcbc57f1066b7a32',
     'name': 'Cultural Center',
     'pluralName': 'Cultural Centers',
     'primary': True,
     'shortName': 'Cultural Center'}],
   'id': '4b17eb00f964a520a1c923e3',
   'location': {'address': 'R. Álvares Penteado, 112',
    'cc': 'BR',
    'city': 'São Paulo',
    'country': 'Brasil',
    'crossStreet': 'R. Quitanda',
    'formattedAddress': ['R. Álvares Penteado, 112 (R. Quitanda)',
     'São Paulo, SP',
     '01012-000'],
    'labeledLatLngs': [{'label': 'display',
      'lat': -23.547588190396358,
      'lng': -46.6346831174672}],
    'lat': -23.547588190396358,
    'lng': -46.6346831174672,
    'postalCode': '01012-

In [ ]:
QUERY = ['Coffee','Café','Cafe']
CATEGORYID = 'bf58dd8d48988d1e0931735' # Coffee Shops ID

url_SP = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&oauth_token={}&v={}&query={}&radius={}&limit={}&near={}".format(
    CLIENT_ID,
    CLIENT_SECRET,
    ACCESS_TOKEN,
    VERSION,
    QUERY,
    RADIUS,
    LIMIT,
    CITY)
Coffee_results = requests.get(url_SP).json()

In [ ]:
coffee_venues = Coffee_results['response']['venues']
coffee_df = pd.json_normalize(coffee_venues)
coffee_df.head(10)

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.cc,location.country,location.formattedAddress,location.crossStreet,location.postalCode,location.city,location.state,venuePage.id,location.neighborhood
0,5054ab84e4b0a9a6e6d65618,Cafe Bancario,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",v-1606353199,False,Lgo.do café,-23.546318,-46.635187,"[{'label': 'display', 'lat': -23.5463182775291...",BR,Brasil,[Lgo.do café],NaN,NaN,NaN,NaN,NaN,NaN
1,59f9d6f08c812a731e820513,CoffeeShop HotelCafe at Nikkey Palace Hotel (C...,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1606353199,False,"R. Galvão Bueno, 425",-23.559383,-46.634981,"[{'label': 'display', 'lat': -23.5593830674518...",BR,Brasil,"[R. Galvão Bueno, 425 (Rua Fagundes), São Paul...",Rua Fagundes,01506-000,São Paulo,SP,459758463,NaN
2,4c004a76f61ea5934eaaeb13,Alhambra Café,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1606353199,False,"R. S. Bento, 344",-23.546367,-46.635249,"[{'label': 'display', 'lat': -23.5463668761844...",BR,Brasil,"[R. S. Bento, 344 (Largo do Café), São Paulo, ...",Largo do Café,01008-010,São Paulo,SP,NaN,NaN
3,4d9b6196e086a35dd113d0ad,Sampa Cafe & Lanches,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",v-1606353199,False,"R. Três de Dezembro, 51",-23.546323,-46.634989,"[{'label': 'display', 'lat': -23.5463225014652...",BR,Brasil,"[R. Três de Dezembro, 51, São Bernardo do Camp...",NaN,09625-050,São Bernardo do Campo,SP,NaN,NaN
4,4c9a5a2b9c66370428ee57fd,Café Martinelli Midi,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1606353199,False,"R. Líbero Badaró, 508",-23.545257,-46.635604,"[{'label': 'display', 'lat': -23.5452570624986...",BR,Brasil,"[R. Líbero Badaró, 508 (Av. São João), São Pau...",Av. São João,01009-000,São Paulo,SP,48036220,NaN
5,5f09da155ee95c76eed14e91,Santander - Work Café Centro,"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",v-1606353199,False,Rua Boa Vista 263,-23.545462,-46.633755,"[{'label': 'display', 'lat': -23.5454623, 'lng...",BR,Brasil,"[Rua Boa Vista 263, São Paulo, SP, 01014-001]",NaN,01014-001,São Paulo,SP,583110239,NaN
6,52d5112d498ea9bad3b07ff7,Café da Bolsa BM&FBovespa,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1606353199,False,Rua João Brícola,-23.545452,-46.634039,"[{'label': 'display', 'lat': -23.5454516259072...",BR,Brasil,"[Rua João Brícola (Praça Antônio Prado), São P...",Praça Antônio Prado,NaN,São Paulo,SP,NaN,NaN
7,4e7786cfd4c0934472cd64c0,Maquina De Café - 12 andar,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1606353199,False,Boa vista,-23.544812,-46.634171,"[{'label': 'display', 'lat': -23.544812, 'lng'...",BR,Brasil,"[Boa vista, São Paulo, SP]",NaN,NaN,São Paulo,SP,NaN,NaN
8,55a7affd498e7f6382b05d97,Fran's Café,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1606353199,False,"Rua Da Quitanda, 130",-23.547726,-46.635815,"[{'label': 'display', 'lat': -23.5477256774902...",BR,Brasil,"[Rua Da Quitanda, 130, São Paulo, SP]",NaN,NaN,São Paulo,SP,NaN,NaN
9,55d7bdc0498e45a2024c4515,Magg Café,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1606353199,False,"Av. Ipiranga, 200, Lj. 75",-23.546383,-46.644088,"[{'label': 'display', 'lat': -23.5463831136759...",BR,Brasil,"[Av. Ipiranga, 200, Lj. 75, São Paulo, SP, 010...",NaN,01046-010,São Paulo,SP,NaN,NaN


In [ ]:
filtered_columns = ['name', 'categories', 'location.neighborhood', 'location.lat', 'location.lng', 'id'] # + [col for col in coffee_df.columns if col.startswith('location.')]
filtered_coffee = coffee_df.loc[:, filtered_columns]
# filtered_coffee.head()

filtered_coffee['categories'] = filtered_coffee.apply(get_category_type, axis=1)
filtered_coffee.columns = [column.split('.')[-1] for column in filtered_coffee.columns]
print(filtered_coffee.shape)
filtered_coffee.head()

(50, 6)


,name,categories,neighborhood,lat,lng,id
0,Cafe Bancario,Diner,NaN,-23.546318,-46.635187,5054ab84e4b0a9a6e6d65618
1,CoffeeShop HotelCafe at Nikkey Palace Hotel (C...,Café,NaN,-23.559383,-46.634981,59f9d6f08c812a731e820513
2,Alhambra Café,Café,NaN,-23.546367,-46.635249,4c004a76f61ea5934eaaeb13
3,Sampa Cafe & Lanches,Breakfast Spot,NaN,-23.546323,-46.634989,4d9b6196e086a35dd113d0ad
4,Café Martinelli Midi,Café,NaN,-23.545257,-46.635604,4c9a5a2b9c66370428ee57fd


In [ ]:
print(filtered_coffee.loc[:,['name', 'categories']])

                                                 name                         categories
0                                       Cafe Bancario                              Diner
1   CoffeeShop HotelCafe at Nikkey Palace Hotel (C...                               Café
2                                       Alhambra Café                               Café
3                                Sampa Cafe & Lanches                     Breakfast Spot
4                                Café Martinelli Midi                               Café
5                        Santander - Work Café Centro                               Bank
6                           Café da Bolsa BM&FBovespa                               Café
7                         Maquina De Café -  12 andar                               Café
8                                         Fran's Café                        Coffee Shop
9                                           Magg Café                        Coffee Shop
10                   

In [ ]:
filtered_coffee.groupby('categories').count()

,name,neighborhood,lat,lng,id
categories,,,,,
Bank,1,0,1,1,1
Bar,1,0,1,1,1
Beer Garden,1,0,1,1,1
Brazilian Restaurant,8,0,8,8,8
Breakfast Spot,1,0,1,1,1
Café,23,1,23,23,23
Coffee Shop,7,0,7,7,7
Diner,2,0,2,2,2
Italian Restaurant,1,0,1,1,1


In [ ]:
address = 'Sao Paulo, SP'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-23.5506507 -46.6333824


In [ ]:
import branca as bc

santander = [-23.551416753127207, -46.63375810307533]

venues_map = folium.Map(location=santander, zoom_start=15)

folium.Marker(
    santander,
    popup='<b>Santander<b>',
    icon=folium.Icon(color='red',
                         prefix='fa',
                         icon='university')
).add_to(venues_map)

for lat, lng, label in zip(filtered_coffee.lat, filtered_coffee.lng, filtered_coffee.name):
  folium.Marker(
      location = [lat, lng],
      popup = label,
      icon = folium.Icon(color='beige',
                       icon_color ='black',
                       prefix='fa',
                       icon='coffee') # Coffee Icon HTML: <i class="fas fa-coffee"></i>
  ).add_to(venues_map)

# display map
venues_map

In [ ]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    nyc_data = json.load(json_data)

Data downloaded!


In [ ]:
neighborhoods_data = nyc_data['features']
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

In [ ]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)